In [9]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None)

In [5]:
df.columns=['views', 'buys']

In [53]:
def to_sparse_matrix(index_, list_of_good):
    return

SyntaxError: invalid syntax (<ipython-input-53-455680ca2399>, line 1)

In [8]:
df.tail()

,views,buys
49995,"32291,60520,32291,38220",32291
49996,"60479,28288,102804,102805",NaN
49997,"980,20025,980,20025,980,20025,980,20025",NaN
49998,"8844,42500,8838,8172,29237,352,8847,6681,8827,...",NaN
49999,"39047,102806,27774",NaN


In [148]:
df_list = df.apply(lambda x: x.str.split(','), axis=0)

In [149]:
df_list['len_v'] = df_list.views.apply(lambda x: np.ones(len(x), dtype=np.int))

In [150]:
df_list['len_v'] = df_list.len_v * df_list.index

In [151]:
df_list

,views,buys,len_v
0,"[0, 1, 2, 3, 4, 5]",NaN,"[0, 0, 0, 0, 0, 0]"
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN,"[1, 1, 1, 1, 1, 1, 1, 1]"
2,"[16, 17, 18, 19, 20, 21]",NaN,"[2, 2, 2, 2, 2, 2]"
3,"[24, 25, 26, 27, 24]",NaN,"[3, 3, 3, 3, 3]"
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]"
...,...,...,...
49995,"[32291, 60520, 32291, 38220]",[32291],"[49995, 49995, 49995, 49995]"
49996,"[60479, 28288, 102804, 102805]",NaN,"[49996, 49996, 49996, 49996]"
49997,"[980, 20025, 980, 20025, 980, 20025, 980, 20025]",NaN,"[49997, 49997, 49997, 49997, 49997, 49997, 499..."
49998,"[8844, 42500, 8838, 8172, 29237, 352, 8847, 66...",NaN,"[49998, 49998, 49998, 49998, 49998, 49998, 499..."


In [152]:
n = np.hstack(df_list.views.to_numpy()).astype(np.int)
b = np.hstack(df_list.len_v.to_numpy()).astype(np.int)


In [153]:
len(n)

356177

In [154]:
len(b)

356177

In [155]:
from scipy.sparse import coo_matrix

In [160]:
coo = coo_matrix((np.ones(len(n), dtype=np.int), (b, n)))

In [176]:
coo.getrow(49998).sum()

10